In [8]:
from transformers import pipeline 
import json

def calculate_top_k_predictions(results, top_k_values=[1, 10, 50]):
    """
    Calcola le predizioni corrette per ciascun modello nelle categorie:
    - Top 1
    - Top 10
    - Top 50
    - Non in Top 50
    
    Args:
    - results: Lista di dizionari con chiavi:
        - `gold_token`: Il token corretto
        - `predicted_tokens`: Lista dei token predetti in ordine di probabilità
    - top_k_values: Lista dei valori di Top-k da analizzare (default: [1, 10, 50])

    Returns:
    - metrics: Dizionario con i conteggi per ogni categoria
    """
    metrics = {
        "top_1": 0,
        "top_10": 0,
        "top_50": 0,
        "not_in_top_50": 0
    }
    
    for result in results:
        gold_token = result["correct_token"]
        predicted_tokens = result["predicted_tokens"]
        
        # Determina la categoria del gold token
        if gold_token == predicted_tokens[0]:
            metrics["top_1"] += 1
        elif gold_token in predicted_tokens[:top_k_values[1]]:
            metrics["top_10"] += 1
        elif gold_token in predicted_tokens[:top_k_values[2]]:
            metrics["top_50"] += 1
        else:
            metrics["not_in_top_50"] += 1

    return metrics


def convert_to_percentages(metrics, total):
    """
    Converte i conteggi delle metriche in percentuali.

    Args:
    - metrics: Dizionario con i conteggi delle metriche.
    - total: Numero totale di frasi.

    Returns:
    - metrics_percent: Dizionario con le percentuali per ogni categoria.
    """
    metrics_percent = {
        key: (value / total) * 100 if total > 0 else 0 for key, value in metrics.items()
    }
    return metrics_percent


def evaluate_models(models, test_data, top_k_values=[1, 10, 50]):
    """
    Esegue le predizioni per ogni modello e calcola le metriche Top-k.

    Args:
    - models: Dizionario di pipeline fill-mask {nome_modello: pipeline_fillmask}.
    - test_data: Lista di dizionari con chiavi:
        - `masked_sentence`: Frase con <mask>
        - `gold_token`: Token corretto
    - top_k_values: Lista dei valori di Top-k da analizzare (default: [1, 10, 50])

    Returns:
    - summary: Dizionario con metriche aggregate per ciascun modello
    """
    summary = {}
    total_sentences = len(test_data)
    
    for model_name, fillmask_pipeline in models.items():
        print(f"Eseguendo predizioni per il modello: {model_name}")
        results = []
        for entry in test_data:
            masked_sent = entry["masked_sentence"]
            if model_name == 'LatinBERT':
                masked_sent = masked_sent.replace("<mask>", "[MASK]")
            gold_token = entry["correct_token"]

            # Predizioni del modello
            predictions = fillmask_pipeline(masked_sent, top_k=max(top_k_values))
            predicted_tokens = [pred["token_str"].strip().replace(" ", "") for pred in predictions]

            # Salva il risultato
            results.append({
                "correct_token": gold_token,
                "predicted_tokens": predicted_tokens
            })

        # Calcola le metriche per il modello corrente
        metrics = calculate_top_k_predictions(results, top_k_values=top_k_values)
        metrics_percent = convert_to_percentages(metrics, total_sentences)
        summary[model_name] = metrics_percent

    return summary



# Modelli da valutare
model_xlmr = "Cicciokr/XLM-Roberta-Base-Latin-Uncased"
model_roberta = "Cicciokr/Roberta-Base-Latin-Uncased"
model_latinbert = "/kaggle/input/bert-base-latin-uncased/transformers/default/1"
model_latincybert = "latincy/latinbert2"

# Crea le pipeline fill-mask per i modelli
models = {
    "XLM-RoBERTa": pipeline("fill-mask", model=model_xlmr, tokenizer=model_xlmr),
    "RoBERTa": pipeline("fill-mask", model=model_roberta, tokenizer=model_roberta),
    "LatinBERT": pipeline("fill-mask", model=model_latinbert, tokenizer=model_latinbert)
}

    # Dataset di test
with open("/kaggle/input/the-latin-library/test_data_latinlibrary.json", "r", encoding="utf-8") as f:
    test_data = json.load(f)

# Esegue la valutazione
summary = evaluate_models(models, test_data)

# Stampa i risultati
print("\n=== Risultati delle Metriche ===")
for model_name, metrics in summary.items():
    print(f"\nModello: {model_name}")
    for metric, value in metrics.items():
        print(f"  {metric}: {value}")


Device set to use cpu
Device set to use cpu
Device set to use cpu


Eseguendo predizioni per il modello: XLM-RoBERTa
Eseguendo predizioni per il modello: RoBERTa


Token indices sequence length is longer than the specified maximum sequence length for this model (131 > 50). Running this sequence through the model will result in indexing errors


Eseguendo predizioni per il modello: LatinBERT

=== Risultati delle Metriche ===

Modello: XLM-RoBERTa
  top_1: 10
  top_10: 16
  top_50: 2
  not_in_top_50: 21

Modello: RoBERTa
  top_1: 18
  top_10: 15
  top_50: 3
  not_in_top_50: 13

Modello: LatinBERT
  top_1: 18
  top_10: 15
  top_50: 3
  not_in_top_50: 13
